In [2]:
!pip install ollama

  Using cached ollama-0.4.7-py3-none-any.whl.metadata (4.7 kB)
Using cached ollama-0.4.7-py3-none-any.whl (13 kB)


In [ ]:
import ollama
import re
from typing import Dict, List, Tuple

class TravelRestrictionsBot:
    def __init__(self, model_name="gemma3:1b"):
        """
        여행 금지 품목 확인 챗봇 초기화
        
        Args:
            model_name: 사용할 Ollama 모델 이름
        """
        self.model_name = model_name
        self.conversation_history = []
        
        # 시스템 메시지 설정 (여행 금지 품목 전문가 역할 부여)
        self.system_message = {
            "role": "system",
            "content": """
            당신은 여행 금지 품목 및 제한사항에 대한 전문적인 지식을 갖춘 챗봇입니다.
            각국의 입국 시 금지/제한되는 품목에 대해 정확한 정보를 제공해야 합니다.
            사용자가 나라를 명시하지 않으면 추가 정보를 요청하세요.
            알 수 없는 정보에 대해서는 솔직하게 모른다고 답변하고, 공식 정보를 확인할 것을 권장하세요.
            
            주요 국가별 일반적인 금지/제한 품목:
            - 일본: 마약류, 총기류, 가짜 상품, 특정 농산물과 육류
            - 미국: 과일, 채소, 육류, 식물, 특정 의약품, 6개월 이상된 식품
            - 중국: 정치적으로 민감한 자료, 종교 관련 자료, 마약류, 도박 관련 물품
            - 태국: 마약류, 담배(200개비 초과), 술(1리터 초과), 일부 의약품
            - 호주: 신선 과일, 채소, 육류, 유제품, 씨앗, 목재 제품
            
            항공기 탑승 시 일반적 제한:
            - 액체류: 개별 용기 100ml 이하, 1L 투명 지퍼백에 담아야 함
            - 예외: 유아식, 의약품 등(보안 검색 시 별도 신고)
            - 위험물: 라이터(1개만 기내 반입 가능), 성냥, 스프레이, 압축 가스
            - 날카로운 물건: 칼, 가위, 면도칼 등은 위탁 수하물로만 가능
            """
        }
    
    def detect_country(self, query: str) -> Tuple[bool, str]:
        """
        사용자 질문에서 국가명을 감지
        
        Args:
            query: 사용자 질문
            
        Returns:
            감지 여부(bool)와 감지된 국가명(없으면 빈 문자열)
        """
        countries = ["일본", "미국", "중국", "태국", "호주", "유럽", "한국", 
                    "영국", "프랑스", "독일", "이탈리아", "스페인", "캐나다", 
                    "싱가포르", "말레이시아", "인도네시아", "베트남", "필리핀"]
        
        for country in countries:
            if country in query:
                return True, country
        
        return False, ""
    
    def detect_item(self, query: str) -> Tuple[bool, str]:
        """
        사용자 질문에서 물품명을 감지
        
        Args:
            query: 사용자 질문
            
        Returns:
            감지 여부(bool)와 감지된 물품명(없으면 빈 문자열)
        """
        items = ["액체", "음식", "과일", "육류", "약", "의약품", "주류", "술", 
                "담배", "화장품", "식품", "전자제품", "배터리", "칼", "가위"]
        
        for item in items:
            if item in query:
                return True, item
        
        return False, ""
    
    def is_travel_restriction_query(self, query: str) -> bool:
        """
        여행 금지 품목 관련 질문인지 확인
        
        Args:
            query: 사용자 질문
            
        Returns:
            여행 금지 품목 관련 질문 여부
        """
        keywords = ["가져갈 수 있", "반입", "금지", "제한", "통관", "세관", 
                   "여행", "입국", "비행기", "기내", "수하물", "짐"]
        
        for keyword in keywords:
            if keyword in query:
                return True
        
        return False
        
    def enhance_prompt(self, query: str) -> str:
        """
        사용자 질문을 분석하여 더 명확한 답변을 유도하는 프롬프트 강화
        
        Args:
            query: 사용자 질문
            
        Returns:
            강화된 프롬프트
        """
        enhanced_query = query
        
        # 국가 감지
        has_country, country = self.detect_country(query)
        
        # 물품 감지
        has_item, item = self.detect_item(query)
        
        # 여행 금지 품목 관련 질문인지 확인
        is_restriction_query = self.is_travel_restriction_query(query)
        
        # 프롬프트 강화
        if is_restriction_query:
            if has_country and has_item:
                enhanced_query = f"{query}\n\n다음에 대해 구체적으로 답변해주세요: {country}으로 여행 시 {item}의 반입 제한에 대한 정보"
            elif has_country and not has_item:
                enhanced_query = f"{query}\n\n{country}으로 여행 시 일반적인 금지/제한 품목에 대해 답변해주세요."
            elif not has_country and has_item:
                enhanced_query = f"{query}\n\n여행 시 {item}의 일반적인 반입 제한에 대해 답변해주세요. 가능하면 주요 국가별로 구분해서 설명해주세요."
            else:
                enhanced_query = f"{query}\n\n여행 시 일반적인 금지/제한 품목에 대해 답변해주세요. 특정 국가를 언급하지 않았으므로 일반적인 국제 여행 기준으로 설명해주세요."
        
        return enhanced_query
    
    def get_response(self, user_query: str) -> str:
        """
        사용자 질문에 대한 응답 생성
        
        Args:
            user_query: 사용자 질문
            
        Returns:
            챗봇 응답
        """
        # 프롬프트 강화
        enhanced_query = self.enhance_prompt(user_query)
        
        # 사용자 메시지 추가
        self.conversation_history.append({
            "role": "user",
            "content": enhanced_query
        })
        
        # 메시지 준비 (시스템 메시지 + 대화 기록)
        messages = [self.system_message] + self.conversation_history
        
        try:
            # Ollama API 호출
            response = ollama.chat(
                model=self.model_name,
                messages=messages
            )
            
            # 응답 추출
            assistant_message = response.message.content
            
            # 대화 기록에 응답 추가
            self.conversation_history.append({
                "role": "assistant",
                "content": assistant_message
            })
            
            # 대화 기록 관리 (너무 길어지면 오래된 대화 삭제)
            if len(self.conversation_history) > 10:
                # 처음 시스템 메시지 이후의 가장 오래된 대화 2개 (user, assistant) 삭제
                self.conversation_history = self.conversation_history[2:]
            
            return assistant_message
            
        except Exception as e:
            error_message = f"오류가 발생했습니다: {str(e)}"
            print(error_message)
            return error_message

def run_test():
    """
    챗봇 테스트 함수
    """
    bot = TravelRestrictionsBot()
    
    # 테스트 질문 목록
    test_questions = [
        "안녕하세요, 처음 뵙겠습니다.",
        "일본 여행 시 가져갈 수 없는 물건이 뭐가 있나요?",
        "비행기에 액체류 가지고 탈 수 있나요?",
        "호주에 과일을 가지고 갈 수 있나요?",
        "태국 여행시 약은 얼마나 가져갈 수 있어요?",
        "면도기를 기내에 가지고 탈 수 있나요?",
        "담배는 얼마나 가져갈 수 있어요?",
        "왜 호주는 음식물 반입이 까다로운가요?"
    ]
    
    print("=== 여행 금지 품목 확인 챗봇 테스트 ===")
    print(f"사용 모델: {bot.model_name}")
    print("=" * 50)
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n[질문 {i}] {question}")
        response = bot.get_response(question)
        print(f"\n[응답 {i}] {response}")
        print("-" * 50)
    
    # 대화형 모드
    print("\n\n=== 대화형 모드 시작 (종료하려면 'q' 또는 'exit' 입력) ===")
    while True:
        user_input = input("\n질문을 입력하세요: ")
        
        if user_input.lower() in ['q', 'exit', '종료']:
            break
            
        response = bot.get_response(user_input)
        print(f"\n[챗봇] {response}")

if __name__ == "__main__":
    run_test()

=== 여행 금지 품목 확인 챗봇 테스트 ===
사용 모델: gemma3:1b

[질문 1] 안녕하세요, 처음 뵙겠습니다.

[응답 1] 안녕하세요! 만나서 반갑습니다. 저는 여행 금지 품목 및 제한사항에 대한 전문적인 지식을 갖춘 챗봇입니다. 어떤 나라에 대한 정보를 찾고 계신가요? 아니면 특정 국가에 대한 문의가 있으신가요? 

궁금하신 점을 자세히 알려주시면 최대한 정확하고 유용한 정보를 제공해 드리겠습니다. 😊
--------------------------------------------------

[질문 2] 일본 여행 시 가져갈 수 없는 물건이 뭐가 있나요?

[응답 2] 일본 여행 시 가져가기 어려울 수 있는 품목들을 자세히 알려드릴게요. 일본은 엄격한 규제와 관습을 가지고 있으니 주의해야 합니다.

**1. 마약류:**

*   **정확히 어떤 종류의 마약류인지 매우 중요합니다.** 일본은 마약류의 제조, 판매, 소지, 운반 등 모든 활동을 엄격하게 금지하고 있습니다.
*   **특정 종류의 약물**은 더욱 엄격하게 제한됩니다. 예를 들어, 특정 종류의 정신과 약물, 성인용 약물, 또는 특정 종류의 약물(예: 항생제, 항암제)은 금지됩니다.
*   **마약류 관련 물품**은 반드시 특별한 신고가 필요하며, 검문 시 엄격한 심문이 진행됩니다.

**2. 총기류:**

*   **총기류**는 일본에서 불법입니다.
*   **총기류 관련 물품**은 특별한 허가를 받아야 하며, 검문 시 매우 엄격한 심문이 진행됩니다.

**3. 가짜 상품:**

*   **가짜 상품**은 일본의 법률에 따라 엄격하게 금지됩니다.
*   **가짜 상품**은 검문 시 심문이 진행될 수 있으며, 심각한 경우 처벌을 받을 수 있습니다.

**4. 특정 농산물 및 육류:**

*   **마약류 농산물 및 육류**는 일본의 법률에 따라 금지됩니다.
*   **특정 종류의 농산물**은 일본의 식량 규제를 준수해야 합니다.

**5. 의약품:**

*   **특정 종류의 의약품*